# Simple Generative App Using Palm 2 & LangChain

## Installation & Authentication

**Install google-generativeai & langchain**
- Get API KEY from MakerSuite or Google Cloud

In [18]:
GOOGLE_API_KEY='APIKey'

In [ ]:
!pip install langchain google-generativeai

**(Or) Authenticate & Initialize Google Cloud Project**

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

PROJECT_ID = "api-project-503433767370"  # @param {type:"string"}

LOCATION = "us-central1"  # @param {type:"string"}


**Install Google Cloud AI Platform and Define Vertex AI Base Model & Class**

In [ ]:
# Install Vertex AI LLM SDK# Install Vertex AI LLM SDK
! pip install google-cloud-aiplatform==1.25.0

from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")


from pydantic import BaseModel, root_validator
from typing import Any, Mapping, Optional, List, Dict
from langchain.llms.base import LLM

class _VertexCommon(BaseModel):
    """Wrapper around Vertex AI large language models.

    To use, you should have the
    ``google.cloud.aiplatform.private_preview.language_models`` python package
    installed.
    """
    client: Any = None #: :meta private:
    model_name: str = "text-bison@001"
    """Model name to use."""

    temperature: float = 0.2
    """What sampling temperature to use."""

    top_p: int = 0.8
    """Total probability mass of tokens to consider at each step."""

    top_k: int = 40
    """The number of highest probability tokens to keep for top-k filtering."""

    max_output_tokens: int = 200
    """The maximum number of tokens to generate in the completion."""

    @property
    def _default_params(self) -> Mapping[str, Any]:
        """Get the default parameters for calling Vertex AI API."""
        return {
            "temperature": self.temperature,
            "top_p": self.top_p,
            "top_k": self.top_k,
            "max_output_tokens": self.max_output_tokens
        }

    def _predict(self, prompt: str, stop: Optional[List[str]]) -> str:
        res = self.client.predict(prompt, **self._default_params)
        return self._enforce_stop_words(res.text, stop)

    def _enforce_stop_words(self, text: str, stop: Optional[List[str]]) -> str:
        if stop:
            return enforce_stop_tokens(text, stop)
        return text

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "vertex_ai"

class VertexLLM(_VertexCommon, LLM):
    model_name: str = "text-bison@001"

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the python package exists in environment."""
        try:
            from vertexai.preview.language_models import TextGenerationModel
        except ImportError:
            raise ValueError(
                "Could not import Vertex AI LLM python package. "
            )

        try:
            values["client"] = TextGenerationModel.from_pretrained(values["model_name"])
        except AttributeError:
            raise ValueError(
                "Could not set Vertex Text Model client."
            )

        return values

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call out to Vertex AI's create endpoint.

        Args:
            prompt: The prompt to pass into the model.

        Returns:
            The string generated by the model.
        """
        return self._predict(prompt, stop)



## Initiatlize Vertex AI

In [3]:
# Initialize Vertex AI SDK
import vertexai
PROJECT_ID = "api-project-503433767370"  # @param {type:"string"}

LOCATION = "us-central1"  # @param {type:"string"}

vertexai.init(project=PROJECT_ID, location=LOCATION)

##Invoking Chat Bison - Raw

*Invoke Chat Bison*

In [21]:

import google.generativeai as palm

palm.configure(api_key=GOOGLE_API_KEY)

defaults = {
  'model': 'models/chat-bison-001',
  'temperature': 0.25,
  'candidate_count': 1,
  'top_k': 40,
  'top_p': 0.95,
}
context = "You are an astronomer, knowledgeable about the solar system."
examples = [
  [
    "What languages do you know",
    "I can speak and translate between over 200 languages"
  ],
  [
    "Write a  poem in English",
    "Sure, here is a poem I wrote in English:\n\nThe world is a beautiful place"
  ]
]
messages = [
  "Write a poem on love in english",
  "**Love's Embrace**\n\nLove is a mystery, a force so deep"
]
messages.append("How many moons does Mars have?")
response = palm.chat(
  **defaults,
  context=context,
  examples=examples,
  messages=messages
)
print(response.last) # Response of the AI to your most recent request

Mars has two moons, Phobos and Deimos. Phobos is the larger of the two moons, and is about 22 kilometers in diameter. Deimos is the smaller moon, and is about 12 kilometers in diameter. Both moons are thought to be captured asteroids, and are very irregularly shaped. They are also very close to Mars, with orbits that take only a few hours to complete. This means that they appear to move very quickly across the Martian sky.


##Invoking Chat Bison Using LangChain Prompt Template

*Building a Prompt Template with LangChain*

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
print(template.format_messages(text='i dont like eating tasty things.'))

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')]


## Invoking Text Bison directly

In [ ]:
llm = VertexLLM(
    model_name='text-bison-32k',
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)


llm('i dont like eating tasty things.')
#llm(template.format_messages(text='i dont like eating tasty things.'))

' There are many reasons why someone might not like eating tasty things. Some people may have a medical condition that makes it difficult or painful to eat, while others may simply not enjoy the taste of food. Still others may have a psychological aversion to food, or may have had a negative experience with food in the past. If you are concerned about your lack of appetite, it is important to talk to your doctor to rule out any underlying medical conditions.'